In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!unzip '/content/drive/MyDrive/maddy/covid-dataset.zip' -d './data'

Streaming output truncated to the last 5000 lines.
  inflating: ./data/train/Normal/Normal-7449.png  
  inflating: ./data/train/Normal/Normal-745.png  
  inflating: ./data/train/Normal/Normal-7450.png  
  inflating: ./data/train/Normal/Normal-7451.png  
  inflating: ./data/train/Normal/Normal-7452.png  
  inflating: ./data/train/Normal/Normal-7453.png  
  inflating: ./data/train/Normal/Normal-7454.png  
  inflating: ./data/train/Normal/Normal-7455.png  
  inflating: ./data/train/Normal/Normal-7456.png  
  inflating: ./data/train/Normal/Normal-7457.png  
  inflating: ./data/train/Normal/Normal-7458.png  
  inflating: ./data/train/Normal/Normal-7459.png  
  inflating: ./data/train/Normal/Normal-746.png  
  inflating: ./data/train/Normal/Normal-7460.png  
  inflating: ./data/train/Normal/Normal-7461.png  
  inflating: ./data/train/Normal/Normal-7462.png  
  inflating: ./data/train/Normal/Normal-7463.png  
  inflating: ./data/train/Normal/Normal-7464.png  
  inflating: ./data/train/Normal/

In [3]:
imageSize = [299,299]

In [4]:
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras.applications import Xception
from glob import glob
import numpy as np
import matplotlib.pyplot as plt

In [5]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
test_datagen =  ImageDataGenerator(rescale = 1./255)

In [6]:
training_set = train_datagen.flow_from_directory(r'./data/train',
                                                 target_size = (299,299),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

test_set = train_datagen.flow_from_directory(r'./data/test',
                                             target_size = (299,299),
                                             batch_size = 32,
                                             class_mode = 'categorical')

Found 26514 images belonging to 4 classes.
Found 2008 images belonging to 4 classes.


In [7]:
xception = Xception(input_shape=imageSize + [3], weights='imagenet',include_top=False)

83683744/83683744 [==============================] - 1s 0us/step


In [8]:
for layers in xception.layers:
  layers.trainable = False

In [9]:
x = Flatten()(xception.output)

In [10]:
prediction = Dense(4, activation='softmax')(x)

In [11]:
model = Model(inputs= xception.input, outputs = prediction)

In [12]:
model.compile(
    loss='categorical_crossentropy',
    optimizer ='adam',
    metrics=['accuracy']
    )

In [13]:
import tensorflow
my_callback = [
    tensorflow.keras.callbacks.EarlyStopping(patience=6)
]

In [14]:
r = model.fit_generator(
    training_set,
    steps_per_epoch=len(training_set),
    epochs=25,

    validation_data=test_set,
    validation_steps=len(test_set),
    callbacks=my_callback
)

<ipython-input-14-27b78306e816>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  r = model.fit_generator(


Epoch 1/25
829/829 [==============================] - 662s 782ms/step - loss: 2.3467 - accuracy: 0.8201 - val_loss: 3.1121 - val_accuracy: 0.7829
Epoch 2/25
829/829 [==============================] - 639s 771ms/step - loss: 2.3239 - accuracy: 0.8560 - val_loss: 3.0068 - val_accuracy: 0.8098
Epoch 3/25
829/829 [==============================] - 641s 773ms/step - loss: 2.1138 - accuracy: 0.8715 - val_loss: 3.0393 - val_accuracy: 0.8317
Epoch 4/25
829/829 [==============================] - 648s 782ms/step - loss: 2.1946 - accuracy: 0.8786 - val_loss: 6.5456 - val_accuracy: 0.7216
Epoch 5/25
829/829 [==============================] - 647s 781ms/step - loss: 2.1457 - accuracy: 0.8865 - val_loss: 4.3600 - val_accuracy: 0.7913
Epoch 6/25
829/829 [==============================] - 644s 776ms/step - loss: 2.0397 - accuracy: 0.8915 - val_loss: 3.5165 - val_accuracy: 0.8376
Epoch 7/25
829/829 [==============================] - 632s 763ms/step - loss: 2.0155 - accuracy: 0.8950 - val_loss: 4.3058 -

In [18]:
model.save('InceptionV3-covid.h5')

In [19]:
import matplotlib.pyplot as plt
import cv2
from tensorflow.keras.utils import load_img,img_to_array
from tensorflow.keras.applications.inception_v3 import decode_predictions
from tensorflow.keras.models import load_model

In [20]:
model = load_model('InceptionV3-covid.h5')

In [22]:
img = './data/test/Lung_Opacity/Lung_Opacity-1.png'
image = load_img(img, target_size=(299,299))
image = img_to_array(image)
image = image.reshape((1,299,299,3))

In [23]:
image.shape

(1, 299, 299, 3)

In [24]:
pred = model.predict(image)

1/1 [==============================] - 1s 1s/step


In [25]:
labels =['pituitary','meningioma','No tumor','gliomas']

In [26]:
import numpy as np
p = np.argmax(pred)

In [27]:
p

1

In [28]:
labels[p]

'Lung_opacity'